In [1]:
import pandas as pd
import openpyxl
import re

In [2]:
file_path = "Olahan Data Survey.xlsx"  # Update this path
data = pd.read_excel(file_path)

In [3]:
organizations = [
    "Mentoring Department (MD)",
    "Student Council (SC)",
    "Club (Universitas)",
    "Organisasi eksternal",
    "Tidak mengikuti organisasi",
    "Unit Kegiatan Mahasiswa (UKM)",
    "Student Representative Board (SRB)",
    "Club (Program Studi)",
    "Student Union (SU)"
]

In [4]:
question_classifications = {
    "Critical Thinking": [
        "Saya lebih bisa menerima informasi yang benar, daripada informasi palsu",
        "Saya memilih untuk mengetahui kebenaran meskipun terkadang tidak sesuai harapan",
        "Saya lebih setuju dengan informasi yang benar, meskipun informasi tersebut bisa saja tidak mendukung posisi saya",
        "Saya dapat menerima pendapat yang berbeda",
        "Saya anggap bukan masalah besar ketika terjadi perbedaan sudut pandang",
        "Saya jarang berselisih dengan orang yang berbeda pendapat dengan saya",
        "Saya akan mencari bukti kuat sebelum memercayai sebuah informasi yang kurang akurat",
        "Ketika menerima sebuah informasi, saya mencari sumber informasi tersebut sebelum memercayainya",
        "Saya seringkali mencari alasan di balik informasi yang saya dapatkan",
        "Saat mempunyai masalah, saya menyusun prioritas permasalahan yang harus diselesaikan terlebih dahulu",
        "Saya akan mencari jalan keluar hingga tuntas ketika menghadapi rintangan",
        "Saya tekun dalam mencari penyelesaian ketika menghadapi masalah",
        "Saya adalah orang yang menyukai pengetahuan baru",
        "Saya adalah orang yang kritis",
        "Saya mampu memahami suatu topik secara mendalam",
        "Saya suka membaca artikel mengenai fenomena terbaru di masyarakat",
        "Saya suka membaca buku untuk menambah pengetahuan",
        "Saya suka mencari tahu topik-topik hangat di masyarakat melalui podcast",
        "Saya kadang memilih memahami pendapat pribadi agar tidak terjadi perpecahan ketika berada dalam situasi perdebatan",
        "Saya dapat memperbaiki pendapat saya agar selaras dengan pendapat orang lain",
        "Terkadang saya berhenti mencari informasi yang ingin saya ketahui demi kebaikan bersama"
    ],
    "Creativity-Innovative": [
        "Saya suka menemukan ide baru untuk membangun sebuah usaha/bisnis",
        "Saya mengajukan ide yang berbeda dari ide teman-teman saya",
        "Saya mampu menemukan hal baru yang belum pernah ada sebelumnya ketika membangun bisnis",
        "Saya memastikan dulu bahwa ide yang akan saya utarakan bersifat efektif",
        "Saya mengidentifikasi fenomena di lapangan sebelum menemukan sebuah ide usaha yang baru",
        "Sebelum menyampaikan sebuah ide, saya pastikan dulu bahwa ide tersebut layak untuk diterapkan",
        "Saya memikirkan langkah konkret untuk menerapkan ide yang akan saya ajukan",
        "Saya berusaha memberikan ide yang praktis",
        "Saya tidak hanya memberikan ide baru yang abstrak, tapi juga dapat diterapkan"
    ],
    "Problem Solving": [
        "Ketika menghadapi masalah, saya akan mencari tahu faktor penyebab terjadinya permasalahan tersebut",
        "Saya akan memahami sebuah tantangan secara menyeluruh sebelum menghadapinya",
        "Saya menentukan tujuan yang ingin dicapai sebelum menyelesaikan permasalahan yang sedang dihadapi",
        "Saya merencanakan berbagai solusi ketika sedang menghadapi masalah",
        "Saya memiliki berbagai cara untuk mengatasi tantangan yang sedang dihadapi",
        "Saya mempertimbangkan solusi yang pernah digunakan untuk mengatasi permasalahan yang saya hadapi saat ini",
        "Setelah menyelesaikan masalah, saya mengevaluasi kembali solusi yang telah saya terapkan",
        "Saya menemukan hal yang perlu saya perbaiki dari solusi yang telah saya terapkan",
        "Saya akan menggunakan strategi yang telah saya terapkan untuk mengatasi tantangan yang sama di masa depan"
    ],
    "Decision Making": [
        "Dalam sebuah permasalahan, saya bisa menemukan inti dari masalah yang harus diselesaikan",
        "Saya mampu menentukan aspek dari masalah yang harus diselesaikan terlebih dahulu",
        "Saya mampu membuat tujuan sebelum mengambil sebuah keputusan",
        "Sebelum memutuskan sesuatu, saya mampu menetapkan berbagai pilihan keputusan",
        "Saya mampu mencari sebanyak mungkin informasi mengenai setiap pilihan keputusan yang akan saya ambil",
        "Saya mampu mengidentifikasi setiap pilihan keputusan yang tersedia",
        "Berdasarkan data yang telah dimiliki, saya mampu mengidentifikasi dampak dari setiap keputusan yang akan saya ambil",
        "Saya mampu memperkirakan dampak dari setiap keputusan yang akan diambil berdasarkan informasi yang telah saya miliki",
        "Saya mampu mengatasi dampak negatif dari setiap keputusan yang akan saya ambil berdasarkan informasi yang telah saya miliki"
    ],
    "Berkomunikasi": [
        "Saya dapat dengan mudah mengutarakan ide atau hal yang ada di pikiran kepada orang lain",
        "Ketika saya ingin memulai pembicaraan dengan orang lain terlebih dahulu, saya dapat menyampaikan pesan dengan jelas",
        "Ketika saya bicara, kalimat yang saya ucapkan sesuai dengan yang ingin saya sampaikan",
        "Saya mampu memusatkan perhatian ketika ada yang sedang berbicara dengan saya",
        "Saya mampu memahami sudut pandang orang lain ketika berbicara dengannya",
        "Saya akan membiarkan orang lain menyelesaikan kalimatnya sebelum saya menanggapinya",
        "Saya mampu mengutarakan hal yang saya rasakan ke orang lain agar ia mampu memahami saya",
        "Ketika saya tidak setuju dengan pendapat orang lain, saya akan menyampaikannya dengan alasan yang relevan",
        "Saya dapat menerima pendapat atau masukan dari orang lain mengenai saya",
        "Saya biasanya dapat tetap tenang berbicara meski ketika orang lain mulai emosional",
        "Saya dapat tetap berpikir jernih mengenai orang lain meskipun saya sedang marah dengannya",
        "Saya dapat menyelesaikan masalah dengan orang lain tanpa dikendalikan oleh emosi negatif"
    ],
    "Leadership": [
        "Saya mengetahui kelebihan saya yang mampu mendukung peran saya sebagai pemimpin",
        "Saya memahami kelemahan diri yang berpotensi menjadi hambatan dalam menjalankan peran saya sebagai pemimpin",
        "Saya paham nilai-nilai pribadi yang dapat saya terapkan di dalam menjalankan peran saya sebagai pemimpin",
        "Saya sadar bahwa suatu tujuan perlu diterapkan dalam memimpin suatu kelompok atau tim",
        "Saya paham tentang pemimpin yang efektif",
        "Saya paham tentang sistem manajemen yang efektif dalam mengatur sebuah tim",
        "Saya mampu menggerakan anggota dalam tim saya agar dapat bekerja untuk mencapai tujuan bersama",
        "Saya mengetahui kemampuan setiap anggota tim saya",
        "Saya berani mempercayakan sebuah tugas bagi setiap anggota tim agar tercapai tujuan bersama",
        "Saya menghargai berbagai pendapat yang berbeda di dalam tim saya",
        "Saya dapat menyelesaikan sebuah konflik di dalam tim saya dengan cara yang sehat",
        "Ketika anggota tim saya memiliki pendapat yang berbeda, saya membuka kesempatan untuk berdiskusi",
        "Saya mampu berinteraksi dengan anggota tim yang memiliki latar belakang berbeda dengan saya",
        "Saya mampu mensinergikan pengalaman yang berbeda dari setiap anggota untuk mengambil keputusan bersama",
        "Saya mampu mengembangkan setiap sudut pandang yang berbeda dari anggota tim untuk kepentingan bersama",
        "Saya menyadari kelebihan tiap anggota, yang dapat digunakan untuk mencapai tujuan bersama",
        "Saya berusaha memiliki hubungan yang berkualitas dengan anggota tim yang saya pimpin",
        "Ketika ada anggota yang berprestasi, saya akan mengapresiasi dan mengakuinya",
        "Saya mampu mengakomodasi kebutuhan setiap anggota"
    ],
    "Initiative": [
        "Saya sudah menentukan target IPK yang akan saya raih ketika saya lulus nanti",
        "Saya telah menerapkan strategi yang konkrit untuk mencapai target IPK yang telah saya tetapkan",
        "Saya sudah menentukan organisasi apa saja yang akan saya ikuti di universitas",
        "Saya suka menanyakan pendapat dosen mengenai performa saya di perkuliahan",
        "Ketika sedang merencanakan karir, saya bisa menemukan hal-hal yang berpotensi menjadi kendala di masa depan",
        "Beberapa kali saya meminta masukan dari teman sekelompok mengenai kinerja saya dalam tim",
        "Ketika mengalami kegagalan, saya akan menemukan cara untuk mengatasi penyebabnya",
        "Saya bisa belajar suatu hal positif dari kritik yang diberikan oleh dosen",
        "Saya bisa dengan mudah mengembalikan semangat saya lagi ketika menghadapi kegagalan"
    ],
    "Learning Performance": [
        "Saya percaya akan kemampuan saya untuk mampu mengikuti proses perkuliahan dengan baik",
        "Performa saya di perkuliahan merupakan tanggungjawab saya pribadi",
        "Saya yakin saya mampu mengerjakan tugas maupun ujian dengan baik di perkuliahan",
        "Saya dapat menyadari seberapa dalam saya memahami sebuah materi dalam satu mata kuliah",
        "Saya selalu berusaha memahami suatu materi dengan lebih mendalam",
        "Saya tahu cara yang tepat bagi saya untuk bisa memahami sebuah materi hingga saya dapat mengaplikasikannya",
        "Saya berusaha mencari makna dari hal-hal yang sedang saya pelajari saat ini",
        "Saya mencari teman yang sesuai dengan gaya belajar saya agar dapat belajar bersama",
        "Melalui aktivitas di perkuliahan, saya mampu belajar untuk mengambil resiko dalam mengambil keputusan bagi masa depan saya",
        "Saya belajar untuk gigih dalam meraih hal yang saya mimpikan di masa depan",
        "Melalui teman belajar, saya mampu mengadopsi cara belajar yang paling efektif untuk saya terapkan pada diri saya sendiri",
        "Saya berbagi mengenai cara yang paling efektif untuk memahami suatu materi dengan teman",
        "Terkadang saya memakai cara belajar teman saya agar dapat efektif memahami suatu materi kuliah",
        "Kesalahan yang pernah saya buat justru memberi saya kesempatan untuk belajar hal baru",
        "Saya memakai kesalahan di masa lalu sebagai bekal untuk menghadapi masa depan",
        "Saya mulai membuat tujuan-tujuan yang ingin saya capai dalam beberapa rentang waktu ke depan"
    ],
    "Ethical Communication": [
        "Ketika berbicara dengan orang lain, saya akan menyampaikan fakta yang ada",
        "Saya berbicara sesuai dengan kebenaran agar dapat dipercaya orang lain",
        "Sedapat mungkin saya tidak berbohong ketika berbicara dengan siapapun",
        "Saya berusaha menggunakan kalimat yang baku dan sopan ketika berbicara dengan dosen/staff",
        "Saya menatap lawan bicara ketika sedang berbicara dengan dosen/staff",
        "Saya tidak menggunakan kalimat yang saya gunakan ke teman saya ketika berbicara dengan dosen/staff",
        "Saya berusaha berkomunikasi sesuai dengan norma ketika berbicara dengan dosen/staff",
        "Saya berusaha berperilaku sesuai dengan norma atau tata laku yang ada",
        "Saya tidak berani melanggar peraturan yang ada agar tidak mendapat hukuman"
    ],
    "Lifelong Learning": [
        "Saya merasa mampu menyampaikan pendapat dengan menggunakan bahasa Indonesia dengan baik",
        "Saya mampu mengutarakan isi pikiran dengan menggunakan bahasa Inggris dengan baik",
        "Saya dapat mengemukakan pendapat saya dengan didukung oleh bahasa non verbal yang relevan",
        "Saya mampu membangun pengetahuan baru dari berbagai informasi di internet",
        "Saya suka bertukar pengetahuan dengan orang lain melalui media sosial",
        "Saya mampu mengambil kesimpulan dari berbagai informasi yang saya dapatkan di internet",
        "Saya dapat mengetahui momen atau situasi yang tepat untuk mempelajari banyak hal baru",
        "Saya tahu kapan waktu yang efektif untuk saya dapat belajar dengan maksimal",
        "Saya akan berusaha mendapatkan ilmu sebanyak-banyaknya jika saya diberi kesempatan untuk belajar",
        "Saya mengetahui etika berperilaku yang sesuai dengan budaya di lingkungan tempat saya berkuliah",
        "Saya berusaha mempelajari budaya di luar kebiasaan saya agar dapat berbaur",
        "Saya menghargai perbedaan yang muncul ketika bertemu dengan orang dari latar belakang budaya yang berbeda"
    ]
}

In [ ]:
def categorize_by_organization(df, organizations, question_classifications):
    categorized_data = {category: [] for category in question_classifications}

    for org in organizations:
        org_df = df[df[org] == 1]
        
        org_categorized_data = {category: [] for category in question_classifications}

        for index, row in org_df.iterrows():
            student_data = {
                'Survey Name': row['nama_survey'],
                'Student Name': row['name'],
                'NIS': row['nis'],
                'Prodi': row['prodi'],
                'Organization': org
            }
            
            for category, questions in question_classifications.items():
                category_data = student_data.copy()
                total_score = 0
                
                for question in questions:
                    for column in df.columns:
                        # Match based on substring or regex pattern (use re for pattern matching)
                        if re.search(re.escape(question), column, re.IGNORECASE):
                            answer = row[column]
                            category_data[question] = answer
                            total_score += answer
                            break
                        
                if total_score > 0:
                    category_data['Grand Total'] = total_score
        
                if category_data != student_data:
                    org_categorized_data[category].append(category_data)

        with pd.ExcelWriter(f'{org}_2024.xlsx') as writer:
            for category, data in org_categorized_data.items():
                df_category = pd.DataFrame(data
                df_category.to_excel(writer, sheet_name=category, index=False)
    
    print("Data has been saved to separate Excel files for each organization.")

In [21]:
categorize_by_organization(data, organizations, question_classifications)

Data has been saved to separate Excel files for each organization.


In [5]:
def process_survey_data_with_organization_single_column(df_path, organizations, question_classifications, output_dir):
    import pandas as pd
    import re
    import os

    # Load the dataset
    df_all = pd.read_excel(df_path, sheet_name=None)  # Read all sheets
    df = pd.concat(df_all.values(), ignore_index=True)  # Combine all sheets
    df_2023 = pd.read_excel(df_path, sheet_name="Data All 2023")  # Read only "Data All 2023"

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    def is_numeric(value):
        """Helper function to check if a value can be treated as numeric."""
        try:
            float(value)
            return True
        except ValueError:
            return False

    def precompute_column_mapping(data_columns, question_classifications):
        """Precompute column-to-question mapping using regex."""
        mapping = {category: {} for category in question_classifications}
        for category, questions in question_classifications.items():
            for question in questions:
                for column in data_columns:
                    if re.search(re.escape(question), column, re.IGNORECASE):
                        mapping[category][question] = column
                        break
        return mapping

    def categorize_and_save(data, column_mapping, output_file):
        categorized_data = {category: [] for category in question_classifications}

        for idx, row in data.iterrows():
            # Log progress every 100 rows
            if idx % 100 == 0:
                print(f"Processing row {idx}...")

            # Collect basic student information
            student_data = {
                'Survey Name': row['nama_survey'],
                'Student Name': row['name'],
                'NIS': row['nis'],
                'Prodi': row['prodi']
            }

            # Combine organizations into a single column
            student_orgs = [org for org in organizations if row.get(org, 0) == 1]
            student_data['Organization'] = ', '.join(student_orgs) if student_orgs else 'None'

            total_score_per_student = 0

            # Categorize answers by question type
            for category, question_mapping in column_mapping.items():
                category_data = student_data.copy()
                category_total_score = 0
                for question, column in question_mapping.items():
                    answer = row.get(column, None)
                    if is_numeric(answer):
                        answer = float(answer)
                        category_data[question] = answer
                        category_total_score += answer
                if category_total_score > 0:
                    category_data['Grand Total'] = category_total_score
                    total_score_per_student += category_total_score
                    categorized_data[category].append(category_data)

        # Save to an Excel file
        with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
            for category, entries in categorized_data.items():
                if entries:
                    pd.DataFrame(entries).to_excel(writer, sheet_name=category, index=False)

    # Precompute the column mapping for efficiency
    column_mapping = precompute_column_mapping(df.columns, question_classifications)

    # Save categorized data for all sheets combined
    # categorize_and_save(df, column_mapping, os.path.join(output_dir, "Categorized_Data_All.xlsx"))

    # Save categorized data for "Data All 2023"
    categorize_and_save(df_2023, column_mapping, os.path.join(output_dir, "Categorized_Data_2023.xlsx"))


In [6]:
input_file = "Olahan Data Survey.xlsx"  # Replace with the actual file path
output_directory = "output_files"

In [7]:
# Process the data
process_survey_data_with_organization_single_column(input_file, organizations, question_classifications, output_directory)

Processing row 0...
Processing row 100...
Processing row 200...
Processing row 300...
Processing row 400...
Processing row 500...
Processing row 600...
Processing row 700...
Processing row 800...
Processing row 900...
Processing row 1000...
